# Using the eScriptorium Connector

This library provides a simple API to access the eScriptorium online platform.

Install with `pip install escriptorium-connector` and import it as follows.

In [ ]:
from src.escriptorium_connector import EscriptoriumConnector
import os
from dotenv import load_dotenv
import io

## Instantiating the connector

It is probably a best practice to store your validation credentials separately. Here we use `python-dotenv` to load from a `.env` file the user's login credentials and the eScriptorium instance address.

The url of the eScriptorium instance is passed to the connector along with the url address for its api and the user's login credentials. The connector will take care of getting the API key for the user and any necessary cookies for asynchronous requests.

In [ ]:
load_dotenv()
url = str(os.getenv('ESCRIPTORIUM_URL'))
api = f'{url}api/'
username = str(os.getenv('ESCRIPTORIUM_USERNAME'))
password = str(os.getenv('ESCRIPTORIUM_PASSWORD'))
escr = EscriptoriumConnector(url, api, username, password)

## Usage

Once instantiated, the connector provides well-documented and convenient methods to interact with your data on eScriptorium.

You might get a list of all your documents.

In [ ]:
my_documents = escr.get_documents()
document_pk = my_documents[0]["pk"]


In [ ]:
print(len(my_documents))
print(my_documents[0])

## Convenience

The connector provides several conveniences.  Perhaps the most broadly useful is automatically following the paging information to acquire the full dataset requested.  Here the request for document parts collects the full list of parts without the need to check if there are further pages of results available.

In [ ]:
document_parts = escr.get_document_parts(document_pk)

In [ ]:
print(len(document_parts))
selected_part = document_parts[1]
print(selected_part)

## Downloading Files

Images can be easily downloaded/uploaded with the connector. Also the exported XML files are available.

In [ ]:
image = escr.get_image(selected_part["image"]["uri"])

In [ ]:
from IPython.display import Image
display(Image(image))

In [ ]:
import zipfile

transcriptions = escr.get_document_transcriptions(document_pk)
page_xmls_zipped = escr.download_part_pagexml_transcription(document_pk, [x["pk"] for x in document_parts[0:4]], transcriptions[0]["pk"])

# the bytes from the connector are from a zip file, so we need to unzip it
zip = zipfile.ZipFile(io.BytesIO(page_xmls_zipped))
page_xmls = [zip.read(x) for x in zip.infolist()]
print(len(page_xmls))

In [ ]:
import xml.dom.minidom as md

# Let's see what it looks like:
for page_xml in page_xmls:
    dom = md.parse(io.BytesIO(page_xml))
    pretty_xml = dom.toprettyxml()
    # remove the weird newline issue:
    pretty_xml = os.linesep.join([s for s in pretty_xml.splitlines() if s.strip()])
    print(pretty_xml)